In [77]:
import numpy as np
import pandas as pd
df=pd.read_csv('taxi_trip_pricing.csv')
df.head()

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.0,Low,Clear,3.56,0.80,0.32,53.82,36.2624
1,47.59,Afternoon,Weekday,1.0,High,Clear,NaN,0.62,0.43,40.57,NaN
2,36.87,Evening,Weekend,1.0,High,Clear,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,Weekday,4.0,Low,NaN,3.48,0.51,0.15,116.81,36.4698
4,NaN,Evening,Weekday,3.0,High,Clear,2.93,0.63,0.32,22.64,15.6180


In [78]:
df.shape

(1000, 11)

In [79]:
df.isna().sum()

Trip_Distance_km         50
Time_of_Day              50
Day_of_Week              50
Passenger_Count          50
Traffic_Conditions       50
Weather                  50
Base_Fare                50
Per_Km_Rate              50
Per_Minute_Rate          50
Trip_Duration_Minutes    50
Trip_Price               49
dtype: int64

In [80]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imputer=IterativeImputer(random_state=42)
num_df=df.select_dtypes(include='number')
imputed_data=imputer.fit_transform(num_df)
imputed_df=pd.DataFrame(imputed_data,columns=num_df.columns)
imputed_df.isna().sum()

Trip_Distance_km         0
Passenger_Count          0
Base_Fare                0
Per_Km_Rate              0
Per_Minute_Rate          0
Trip_Duration_Minutes    0
Trip_Price               0
dtype: int64

In [81]:
imputed_df.shape

(1000, 7)

In [82]:
cat_col=df.select_dtypes(include='object')
df=pd.concat([imputed_df,cat_col], axis=1)

In [83]:
df['Day_of_Week'].value_counts()

Day_of_Week
Weekday    655
Weekend    295
Name: count, dtype: int64

In [84]:
df['Weather'].value_counts()

Weather
Clear    667
Rain     227
Snow      56
Name: count, dtype: int64

In [85]:
df['Traffic_Conditions'].value_counts()

Traffic_Conditions
Low       397
Medium    371
High      182
Name: count, dtype: int64

In [86]:
df['Time_of_Day'].value_counts()

Time_of_Day
Afternoon    371
Morning      283
Evening      203
Night         93
Name: count, dtype: int64

In [87]:
# cat_cols=cat_col.columns
# df=pd.get_dummies(df,columns=cat_cols,drop_first=True,dtype=int)

In [88]:
df.isna().sum()
df.dropna(inplace=True)

In [89]:
df['Day_of_Week']=df['Day_of_Week'].map({'Weekday':0,'Weekend':1})
df['Weather']=df['Weather'].map({'Clear':0,'Rain':1,'Snow':2})
df['Traffic_Conditions']=df['Traffic_Conditions'].map({'Low':0,'Medium':1,'High':2,'Jam':3})
df['Time_of_Day']=df['Time_of_Day'].map({'Morning':0,'Afternoon':1,'Evening':2,'Night':3})

In [91]:
df[cat_col.columns].astype('int')

,Time_of_Day,Day_of_Week,Traffic_Conditions,Weather
0,0,0,0,0
1,1,0,2,0
2,2,1,2,0
4,2,0,2,0
5,1,1,1,0
...,...,...,...,...
995,1,1,1,0
996,3,0,1,0
997,0,0,0,1
998,0,0,0,0


In [92]:
df.shape

(811, 11)

In [93]:
df

,Trip_Distance_km,Passenger_Count,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price,Time_of_Day,Day_of_Week,Traffic_Conditions,Weather
0,19.350000,3.0,3.560000,0.800000,0.320000,53.82,36.262400,0,0,0,0
1,47.590000,1.0,3.517427,0.620000,0.430000,40.57,79.047866,1,0,2,0
2,36.870000,1.0,2.700000,1.210000,0.150000,37.27,52.903200,2,1,2,0
4,19.722094,3.0,2.930000,0.630000,0.320000,22.64,15.618000,2,0,2,0
5,8.640000,2.0,2.550000,1.710000,0.480000,89.33,60.202800,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
995,5.490000,4.0,2.390000,0.620000,0.490000,58.39,34.404900,1,1,1,0
996,45.950000,4.0,3.120000,0.610000,0.248435,61.96,62.129500,3,0,1,0
997,7.700000,3.0,2.080000,1.780000,0.302450,54.18,33.123600,0,0,0,1
998,47.560000,1.0,2.670000,0.820000,0.170000,114.94,61.209000,0,0,0,0


In [94]:
df.duplicated().sum()

np.int64(0)

In [95]:
from sklearn.feature_selection import VarianceThreshold
numeric_df=df.select_dtypes(include='number')
constant_filter=VarianceThreshold(threshold=0)
constant_filter.fit(numeric_df)
constant_cols=[column for column,var in zip(numeric_df.columns, constant_filter.variances_) if var == 0]
constant_cols

[]

In [96]:
low_var=VarianceThreshold(threshold=0.01)
low_var.fit(numeric_df)
low_var=[column for column,var in zip(numeric_df.columns, low_var.variances_) if var<0.01]
low_var

[]

In [97]:
categorical_df=df.select_dtypes(include='object')
constant_cats=[col for col in categorical_df.columns if categorical_df[col].nunique() == 1]
constant_cats

[]

In [98]:
df.to_csv('Cleaned Data.csv',index=False)